# Music Recomendations

In [1]:
!pip install catboost==1.0.3
!pip install gensim==4.1.0
!pip install --upgrade numpy

     |████████████████████████████████| 12.8 MB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 9.4 MB/s eta 0:00:011
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
^C
ERROR: Operation cancelled by user
  Using cached numpy-1.21.4-cp38-cp38-macosx_10_9_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 0.26.1 requires graphviz, which is not installed.


In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [3]:
%cd /content
!wget https://voudy-data.s3.eu-north-1.amazonaws.com/wsdm_data.tar.gz
!tar -xf /content/wsdm_data.tar.gz
!mkdir data
!mv -t data members.csv song_extra_info.csv songs.csv train.csv
!git clone -b hw2 https://github.com/Sushentsev/recommendation-systems
%cd recommendation-systems/
!mv /content/data /content/recommendation-systems/hw2

[Errno 2] No such file or directory: '/content'
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2
zsh:1: command not found: wget
tar: Error opening archive: Failed to open '/content/wsdm_data.tar.gz'
mkdir: data: File exists
mv: illegal option -- t
usage: mv [-f | -i | -n] [-v] source target
       mv [-f | -i | -n] [-v] source ... directory
Cloning into 'recommendation-systems'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 82 (delta 36), reused 66 (delta 25), pack-reused 0
Receiving objects: 100% (82/82), 5.96 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/recommendation-systems
mv: rename /content/data to /content/recommendation-systems/hw2: No such file or directory


In [4]:
%cd /content/recommendation-systems/hw2

[Errno 2] No such file or directory: '/content/recommendation-systems/hw2'
/Users/Denis.Sushentsev/Study/recommendation-systems/hw2/recommendation-systems


In [5]:
import sys
sys.path.append("/content/recommendation-systems")

In [1]:
from datasets.members import MembersDataset
from datasets.songs import SongsDataset
from datasets.train import TrainDataset
from datasets.songs_info import SongsInfoDataset

from models.catboost_model import CatBoostModel
from models.embeddings_model import EmbeddingModel

from embeddings_builder import EmbeddingsBuilder

from plot import * 
from utils import * 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

## Подготовка данных

In [2]:
random_state = 42

Загрузим данные в память. 

In [3]:
%%time
members_dataset = MembersDataset.from_path("./data/members.csv")
songs_dataset = SongsDataset.from_path("./data/songs.csv")
train_dataset = TrainDataset.from_path("./data/train.csv")
songs_info_dataset = SongsInfoDataset.from_path("./data/song_extra_info.csv")

CPU times: user 55.5 s, sys: 1.27 s, total: 56.7 s
Wall time: 56.9 s


Построим признаки для песен и пользователей и посмотрим, что получилось. 

In [4]:
%%time
members_dataset = members_dataset.create_features()
songs_dataset = songs_dataset.create_features(songs_info_dataset)

CPU times: user 11.3 s, sys: 512 ms, total: 11.8 s
Wall time: 11.9 s


In [5]:
members_dataset.pandas_df.head(3)

,msno,city,gender,registered_via,bd_category,registration_init_year,expiration_date_year
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,<UNK>,7,<UNK>,2011,2017
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,<UNK>,7,<UNK>,2015,2017
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,<UNK>,4,<UNK>,2016,2017


In [6]:
songs_dataset.pandas_df.head(3)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,1,1,1,1,2014
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,1,1,3,1,-1
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,<UNK>,<UNK>,31.0,1,1,0,0,-1


Теперь добавим в обучающую выборку признаков из песен и пользователей. 

In [7]:
train_dataset = train_dataset.merge(members_dataset, on="msno", how="left")
train_dataset = train_dataset.merge(songs_dataset, on="song_id", how="left")

In [8]:
train_dataset.pandas_df.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,gender,registered_via,bd_category,...,genre_ids,artist_name,composer,lyricist,language,genres_count,artist_name_count,composer_count,lyricists_count,isrc_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,<UNK>,7,<UNK>,...,359,Bastille,Dan Smith| Mark Crew,<UNK>,52.0,1.0,1.0,2.0,0.0,2016
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Various Artists,<UNK>,<UNK>,52.0,1.0,1.0,0.0,0.0,1999
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,female,9,young,...,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,<UNK>,52.0,1.0,1.0,1.0,0.0,2006


Теперь преобразуем некоторые колонки категориальным значениям и заполнил пропуски. 

In [9]:
train_dataset.to_category(["msno", "song_id"])
train_dataset.fill_na_category(["source_system_tab", "source_screen_name", "source_type"])

In [10]:
train_dataset.pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   msno                    category
 1   song_id                 category
 2   source_system_tab       category
 3   source_screen_name      category
 4   source_type             category
 5   target                  int64   
 6   city                    category
 7   gender                  category
 8   registered_via          category
 9   bd_category             category
 10  registration_init_year  int64   
 11  expiration_date_year    int64   
 12  song_length             float64 
 13  genre_ids               category
 14  artist_name             category
 15  composer                category
 16  lyricist                category
 17  language                category
 18  genres_count            float64 
 19  artist_name_count       float64 
 20  composer_count          float64 
 21  lyricist

Все круто!

In [11]:
na_mask = train_dataset.pandas_df.isna().any(axis=1)
print(f"Количество записей, для которых не нашлось пользователя или песни: {sum(na_mask)}")

Количество записей, для которых не нашлось пользователя или песни: 114


Их оказалось не так много, уберем из нашего набора данных.

In [12]:
train_dataset = train_dataset.remove_by_mask(na_mask)

Так как **CatBoost** на CPU обучается достаточно долго (больше часа), то было решено обучать на GPU. Однако GPU не поддерживает группы больше 1023, поэтому тренировочный набор данных был уменьшен. При этом, желательно сохранить порядок записей.

In [13]:
train_dataset_sm = train_dataset.reduce_by_members(size=1023)

In [14]:
train_dataset_sm.pandas_df.target.value_counts()

1    3564182
0    3451109
Name: target, dtype: int64

Соотношение классов осталось тем же, все хорошо. Изменение размеров: 7377304 -> 7015291

In [15]:
print(f"{len(train_dataset)} -> {len(train_dataset_sm)}")

7377304 -> 7015291


## Задание 1

Построить рекомендации для каждого пользователя, произвести оценку качества с помощью 5-fold CV с метриками NDCG, ROC AUC per user.

При разработке признаков я умеренно не стал брать признаки, полученные с помощью матричных факторизаций, чтобы потом добавить неклассические эмбеддинги, полученные в задании 2. 

Для **CatBoost** необоходимо, чтобы данные были упорядочены по **queries**, поэтому будем обучать модели на упорядоченных данных, но эмбеддинги составлять на основе натурального порядка. 

In [16]:
train_dataset_sm_sort = train_dataset_sm.sort_by("msno")
model = CatBoostModel("YetiRank", 2, "CPU", random_state)
scores = model.cv_scores(train_dataset_sm_sort, n_splits=5)

Groupwise loss function. OneHotMaxSize set to 10
0:	total: 14.8s	remaining: 14.8s
1:	total: 29.1s	remaining: 0us
Query dcg: 41.64401211378033 | Query idcg: 46.52718972524379
Query dcg: 25.4912201410258 | Query idcg: 26.86265236944712
Query dcg: 15.54536259203795 | Query idcg: 17.233558322025694
Query dcg: 48.4679753617969 | Query idcg: 53.2493064909825
Query dcg: 18.501640541487355 | Query idcg: 23.301005670715348
Query dcg: 40.81997104092188 | Query idcg: 40.83585442437182
Query dcg: 22.170639119648477 | Query idcg: 23.446298691554592
Query dcg: 46.94504587951408 | Query idcg: 53.48578637660232
Query dcg: 66.53856713403322 | Query idcg: 70.66826315252459
Query dcg: 11.235670389104202 | Query idcg: 12.89773270186745
Query dcg: 81.08831214139065 | Query idcg: 85.42177959742745
Query dcg: 6.620499603205211 | Query idcg: 9.361581041840884
Query dcg: 1.803571487760653 | Query idcg: 2.5616063116448506
Query dcg: 10.19302818069242 | Query idcg: 12.007070295381267
Query dcg: 1.509031854842203

Query dcg: 4.996271569368227 | Query idcg: 5.611347998013804
Query dcg: 30.410137860415517 | Query idcg: 32.046309440183045
Query dcg: 1.2563284295394133 | Query idcg: 2.9484591188793923
Query dcg: 0.6560469745624758 | Query idcg: 2.1309297535714578
Query dcg: 7.99624898808853 | Query idcg: 9.361581041840884
Query dcg: 20.851103062365954 | Query idcg: 21.536277476163644
Query dcg: 5.052750456583919 | Query idcg: 5.355389973203989
Query dcg: 12.581041206446347 | Query idcg: 13.42150653836889
Query dcg: 13.92886188616888 | Query idcg: 18.181359140352214
Query dcg: 40.69737750060612 | Query idcg: 40.83585442437182
Query dcg: 20.22404317679774 | Query idcg: 22.129038549102457
Query dcg: 48.56383135783132 | Query idcg: 49.91355226727045
Query dcg: 0.3333333333333333 | Query idcg: 1.0
Query dcg: 49.73484038485388 | Query idcg: 51.587394652437396
Query dcg: 32.71729128609304 | Query idcg: 37.02193404896107
Query dcg: 33.07401952082284 | Query idcg: 35.20943776725005
Query dcg: 16.531055379898

Query dcg: 2.0506711379627385 | Query idcg: 2.9484591188793923
Query dcg: 2.231706553737374 | Query idcg: 2.5616063116448506
Query dcg: 22.083488837895743 | Query idcg: 23.736118107863362
Query dcg: 1.9887107053629989 | Query idcg: 3.637999639320748
Query dcg: 19.10153482521022 | Query idcg: 23.591335157249404
Query dcg: 10.091178741890007 | Query idcg: 10.904380279414635
Query dcg: 3.564003622861399 | Query idcg: 6.5812199200668
Query dcg: 4.307068936683117 | Query idcg: 6.10599854013203
Query dcg: 0.31546487678572877 | Query idcg: 1.0
Query dcg: 2.346631482103217 | Query idcg: 2.9484591188793923
Query dcg: 5.665875740123232 | Query idcg: 7.703681227584122
Query dcg: 0.6185783827825007 | Query idcg: 1.6309297535714575
Query dcg: 67.4754068252945 | Query idcg: 69.21642109362156
Query dcg: 13.184599502917246 | Query idcg: 15.614975746711938
Query dcg: 7.276577691353344 | Query idcg: 9.361581041840884
Query dcg: 6.240100541584389 | Query idcg: 7.040268381923512
Query dcg: 67.772779313145

Query dcg: 82.85742644896014 | Query idcg: 84.3470772732217
Query dcg: 21.527969023311613 | Query idcg: 23.736118107863362
Query dcg: 39.68137752654535 | Query idcg: 40.71013462051254
Query dcg: 6.229702684789336 | Query idcg: 6.345811006700162
Query dcg: 32.42687457959015 | Query idcg: 37.66395877305022
Query dcg: 7.495141542949247 | Query idcg: 9.559820905011444
Query dcg: 6.022741773415304 | Query idcg: 7.040268381923512
Query dcg: 16.369890037503875 | Query idcg: 17.073986906355756
Query dcg: 2.734957075289579 | Query idcg: 5.092740438166795
Query dcg: 68.98606749336109 | Query idcg: 80.23843881598764
Query dcg: 19.684892848254627 | Query idcg: 20.33594298168059
Query dcg: 35.304649628339014 | Query idcg: 38.04789644938416
Query dcg: 22.172989694144768 | Query idcg: 24.456332265627772
Query dcg: 21.447620275705265 | Query idcg: 22.129038549102457
Query dcg: 57.7552293780367 | Query idcg: 60.02020163269607
Query dcg: 52.21124289373769 | Query idcg: 54.78238428649342
Query dcg: 5.509

Query dcg: 24.154246413731066 | Query idcg: 27.281068686012205
Query dcg: 51.07034957121612 | Query idcg: 55.25221424063278
Query dcg: 7.116013098833875 | Query idcg: 7.264512206141087
Query dcg: 7.161074330944608 | Query idcg: 9.559820905011444
Query dcg: 17.11251453854865 | Query idcg: 18.804637947889162
Query dcg: 3.987403758258484 | Query idcg: 5.092740438166795
Query dcg: 1.6046797869879976 | Query idcg: 3.953464516106477
Query dcg: 10.993864378961913 | Query idcg: 13.24774119549745
Query dcg: 24.91439656251548 | Query idcg: 25.312808952881337
Query dcg: 1.6878828028985229 | Query idcg: 2.1309297535714578
Query dcg: 2.2288075278687747 | Query idcg: 3.637999639320748
Query dcg: 6.492291530749215 | Query idcg: 6.5812199200668
Query dcg: 9.53833737703452 | Query idcg: 12.89773270186745
Query dcg: 35.07100449238115 | Query idcg: 37.79204312555925
Query dcg: 15.70562856346528 | Query idcg: 16.43078950846747
Query dcg: 26.743736826418637 | Query idcg: 29.756886348463155
Query dcg: 38.18

...

Query dcg: 2.5616063116448506 | Query idcg: 2.5616063116448506
Query dcg: 17.477039344984764 | Query idcg: 19.267912215380306
Query dcg: 58.9538597783874 | Query idcg: 60.480965765100485
Query dcg: 6.314152028212209 | Query idcg: 6.5812199200668
Query dcg: 14.454037398272105 | Query idcg: 21.981288435923773
Query dcg: 18.004284551873457 | Query idcg: 19.727789458625683
Query dcg: 18.368061901474285 | Query idcg: 22.423681288010684
Query dcg: 2.5616063116448506 | Query idcg: 2.5616063116448506
Query dcg: 24.639618036608404 | Query idcg: 29.348157228258863
Query dcg: 49.04982184660396 | Query idcg: 54.31167657972717
Query dcg: 8.174134701665702 | Query idcg: 9.161581041840885
Query dcg: 33.42804988396243 | Query idcg: 35.4697531309187
Query dcg: 24.269808712878152 | Query idcg: 25.87933106582094
Query dcg: 68.33796944168343 | Query idcg: 74.657901187501
Query dcg: 20.412007350513097 | Query idcg: 21.981288435923773
Query dcg: 17.34457189349825 | Query idcg: 18.959450330624932
Query dcg: 

Query dcg: 46.774203277222576 | Query idcg: 48.22707704067915
Query dcg: 21.921738683236292 | Query idcg: 25.596503573098232
Query dcg: 25.880646480300037 | Query idcg: 26.582722459358394
Query dcg: 42.036532764798466 | Query idcg: 42.95945608046186
Query dcg: 7.246167800290275 | Query idcg: 11.276481714068948
Query dcg: 2.5266626691183243 | Query idcg: 4.254494511770458
Query dcg: 9.414658971327706 | Query idcg: 10.527278095188658
Query dcg: 2.0675293653079345 | Query idcg: 2.5616063116448506
Query dcg: 73.2531348110816 | Query idcg: 83.48540135267955
Query dcg: 15.759363444883457 | Query idcg: 16.913933833100934
Query dcg: 5.420611313295528 | Query idcg: 6.5812199200668
Query dcg: 17.305337954307184 | Query idcg: 17.551292088810584
Query dcg: 15.721860279512484 | Query idcg: 18.49381799371651
Query dcg: 11.821619762331274 | Query idcg: 15.614975746711938
Query dcg: 28.971032610569907 | Query idcg: 30.299579945350697
Query dcg: 20.839111075752424 | Query idcg: 22.71720494211644
Query 

Query dcg: 10.880767313131033 | Query idcg: 12.007070295381267
Query dcg: 78.45835437093761 | Query idcg: 83.80873129504653
Query dcg: 2.6988165387938374 | Query idcg: 4.254494511770458
Query dcg: 6.278896978233377 | Query idcg: 6.812598133226559
Query dcg: 33.248224920471806 | Query idcg: 34.425591754196596
Query dcg: 47.34144085815229 | Query idcg: 52.419800704200625
Query dcg: 1.206418282850453 | Query idcg: 2.5616063116448506
Query dcg: 5.14269393326915 | Query idcg: 5.611347998013804
Query dcg: 3.5219461571418624 | Query idcg: 4.543559338088346
Query dcg: 43.7872411606109 | Query idcg: 44.07376897639696
Query dcg: 0.6309297535714575 | Query idcg: 1.0
Query dcg: 16.452813177870063 | Query idcg: 17.867204118143228
Query dcg: 34.81021752600816 | Query idcg: 34.818064666459115
Query dcg: 3.7768788182340707 | Query idcg: 7.040268381923512
Query dcg: 2.404601483424634 | Query idcg: 2.9484591188793923
Query dcg: 9.964896785005445 | Query idcg: 10.904380279414635
Query dcg: 6.301676082596

Query dcg: 8.048687298013927 | Query idcg: 10.716478454705527
Query dcg: 80.9843332775604 | Query idcg: 86.60094244988082
Query dcg: 31.66826226386772 | Query idcg: 32.97673438544415
Query dcg: 74.21016011752118 | Query idcg: 76.52699413903969
Query dcg: 9.271581462987962 | Query idcg: 10.144767962755324
Query dcg: 52.160713541257046 | Query idcg: 55.36953608391326
Query dcg: 7.390759034629033 | Query idcg: 8.755936908168279
Query dcg: 4.895095845245462 | Query idcg: 5.355389973203989
Query dcg: 19.688487025795006 | Query idcg: 20.93867087428094
Query dcg: 4.5859948233178125 | Query idcg: 4.8225022837394755
Query dcg: 12.853521897632673 | Query idcg: 15.779247796673989
Query dcg: 3.2790442404774005 | Query idcg: 4.254494511770458
Query dcg: 6.747879633706838 | Query idcg: 9.361581041840884
Query dcg: 0.3333333333333333 | Query idcg: 1.0
Query dcg: 8.903705541515674 | Query idcg: 14.108820542845699
Query dcg: 21.95012126764809 | Query idcg: 27.141791156525002
Query dcg: 0.84918024522184

Query dcg: 17.230496731667724 | Query idcg: 19.880354329226844
Query dcg: 1.0 | Query idcg: 1.0
Query dcg: 1.6309297535714575 | Query idcg: 1.6309297535714575
Query dcg: 27.716592819205353 | Query idcg: 28.250733878009168
Query dcg: 0.6821890751874113 | Query idcg: 2.1309297535714578
Query dcg: 7.074883611665916 | Query idcg: 7.264512206141087
Query dcg: 5.2366359516625405 | Query idcg: 6.812598133226559
Query dcg: 11.870178922407344 | Query idcg: 12.72144126747857
Query dcg: 0.23981246656813146 | Query idcg: 1.0
Query dcg: 11.206040241482107 | Query idcg: 15.779247796673989
Query dcg: 40.506085666758686 | Query idcg: 41.713353316517875
Query dcg: 48.28110977996488 | Query idcg: 50.15342888675686
Query dcg: 11.940876996593913 | Query idcg: 12.544257447343012
Query dcg: 37.771071843060064 | Query idcg: 37.92002213796416
Query dcg: 17.925870737043713 | Query idcg: 22.129038549102457
Query dcg: 18.728538641263018 | Query idcg: 25.170630670730326
Query dcg: 6.433152494909259 | Query idcg: 

## Задание 2

Построить неклассические эмбеддинги и исследовать их полезность.

In [ ]:
%%time
embeddings = EmbeddingsBuilder(embedding_dim=100, random_state=random_state)
embeddings = embeddings.fit(train_df)
save_pickle(embeddings, "embeddings.pkl")